<a href="https://colab.research.google.com/github/tarunku/open_llm/blob/main/How_to_Build_LangChain_Agents_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**let’s launch the xterm terminal within our Colab cell**

In [ ]:
!pip install colab-xterm
!curl https://ollama.ai/install.sh | sh
%load_ext colabxterm

In [ ]:
%xterm
 # ollama serve &

In [ ]:
!ollama pull llama3
!ollama pull mistral

In [ ]:
!ollama list

In [ ]:
!ollama --version

In [ ]:
!pip install -qU langchain langchain_community langchain-ollama langgraph wikipedia youtube_search

In [ ]:
from langchain_community.tools import WikipediaQueryRun  # pip install wikipedia
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool  # pip install youtube_search
from langchain_community.tools.openai_dalle_image_generation import (
   OpenAIDALLEImageGenerationTool
)
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper

from langchain import HuggingFaceHub
from langchain.llms import HuggingFacePipeline

import os

from langchain.llms import Ollama
from langchain.chat_models import ChatOllama

from IPython.display import Image, display, YouTubeVideo
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")


In [ ]:
# Set up the OpenAI model,
llm = ChatOllama(model="llama3")

In [ ]:
#Tools

import ast
import re

def wikipedia(toic:str):
  print("wikipedia**************")
  wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
  wikipedia = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
  return wikipedia.invoke(toic)


def dalle(topic:str):
  dalle_api_wrapper = DallEAPIWrapper(model="dall-e-3", size="1024x1024")

  dalle = OpenAIDALLEImageGenerationTool(
    api_wrapper=dalle_api_wrapper
  )

  image_url = dalle.invoke(topic)
  return image_url

def youtube(topic:str):
  youtube = YouTubeSearchTool()
  results = youtube.run(topic)
  results_list = ast.literal_eval(results)
  return results_list


In [ ]:
#Agent
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor
)
from langchain import hub

def agent_007(topic: str) -> str:

    template = """
   You are a helpful bot named Chandler. Your task is to explain topic {topic_name}
   asked by the user via three mediums: text, image or video.

   If the asked topic is best explained in text format, use the Wikipedia tool.
   If the topic is best explained by showing a picture of it, generate an image
   of the topic using Dall-E image generator and print the image URL.
   Finally, if video is the best medium to explain the topic, conduct a YouTube search on it
   and return found video links.
   """

    prompt_template = PromptTemplate(
        template=template, input_variables=["topic_name"]
    )

    tools_for_agent = [
        Tool(
            name="Wikipedia tool",
            func=wikipedia,
            description="A tool to explain things in text format. Use this tool if you think the user’s asked concept is best explained through text.",
        ),
        Tool(
            name="Dall-E image generator",
            func=dalle,
            description="A tool to generate images. Use this tool if you think the user’s asked concept is best explained through an image.",
        ),
        Tool(
            name="Youtube Search tool",
            func=youtube,
            description="A tool to search YouTube videos. Use this tool if you think the user’s asked concept can be best explained by watching a video.",
        )
    ]

    react_prompt = hub.pull("hwchase17/react")

    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)


    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(topic_name=topic)}
    )

    return result["output"]

In [ ]:
agent_007('what is the most famous Lata mangeskar song on youtube?')

In [ ]:
agent_007('a video help on how to ride bicycle?')